# YOUR PROJECT TITLE

Imports and set magics:

In [40]:
# uncomment below if you need to install 
#%pip install git+https://github.com/alemartinello/dstapi
#%pip install pandas-datareader
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')

import pandas_datareader # install with `pip install pandas-datareader`
from dstapi import DstApi # install with `pip install git+https://github.com/alemartinello/dstapi`

# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# user written modules
import Oliver




The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Read and clean data

In [60]:
tabsum['variable name']

0         OMRÅDE
1         MOHERK
2       MOOPRIND
3         MOSTAT
4    MODERSALDER
5        BARNKON
6            Tid
Name: variable name, dtype: object

In [61]:
# Load data
ind = DstApi('FODIE') 
tabsum = ind.tablesummary(language='en')

# Quick overview of data. 
for variable in tabsum['variable name']:
#    print(variable+':')
    display(ind.variable_levels('OMRÅDE', language='en').head(5))


Table FODIE: Live births by region, mothers ancestry, mothers country of origin, mothers citizenship, age of mother, sex of child and time
Last update: 2023-02-10T08:00:00


,id,text
0,000,All Denmark
1,084,Region Hovedstaden
2,101,Copenhagen
3,147,Frederiksberg
4,155,Dragør


,id,text
0,5,Persons of Danish origin
1,4,Immigrants
2,3,Descendant
3,0,Unknown origin


,id,text
0,5100,Denmark
1,5122,Albania
2,5124,Andorra
3,5706,Belarus
4,5126,Belgium


,id,text
0,5100,Denmark
1,5122,Albania
2,5124,Andorra
3,5706,Belarus
4,5126,Belgium


,id,text
0,10,10 years
1,11,11 years
2,12,12 years
3,13,13 years
4,14,14 years


,id,text
0,D,Boys
1,P,Girls


,id,text
0,2007,2007
1,2008,2008
2,2009,2009
3,2010,2010
4,2011,2011


In [54]:
# The _define_base_params -method gives us a nice template (selects all available data)
params = ind._define_base_params(language='en')
params

{'table': 'fodie',
 'format': 'BULK',
 'lang': 'en',
 'variables': [{'code': 'OMRÅDE', 'values': ['*']},
  {'code': 'MOHERK', 'values': ['*']},
  {'code': 'MOOPRIND', 'values': ['*']},
  {'code': 'MOSTAT', 'values': ['*']},
  {'code': 'MODERSALDER', 'values': ['*']},
  {'code': 'BARNKON', 'values': ['*']},
  {'code': 'Tid', 'values': ['*']}]}

In [56]:
inc_api = ind.get_data(params=params)

,OMRÅDE,MOHERK,MOOPRIND,MOSTAT,MODERSALDER,BARNKON,TID,INDHOLD
300530,Aabenraa,Descendant,Germany,Denmark,17 years,Boys,2007,1
280142,Aabenraa,Persons of Danish origin,Denmark,Denmark,18 years,Boys,2007,2
283911,Aabenraa,Immigrants,Armenia,Armenia,18 years,Girls,2007,1
296304,Aabenraa,Persons of Danish origin,Denmark,Denmark,18 years,Girls,2007,2
300536,Aabenraa,Immigrants,Iraq,Iraq,18 years,Boys,2007,1


In [58]:
inc_api.sort_values(by=['TID', 'OMRÅDE', 'MODERSALDER', 'MOOPRIND', 'BARNKON'], inplace=True)
inc_api.head(5)

,OMRÅDE,MOHERK,MOOPRIND,MOSTAT,MODERSALDER,BARNKON,TID,INDHOLD
300530,Aabenraa,Descendant,Germany,Denmark,17 years,Boys,2007,1
283911,Aabenraa,Immigrants,Armenia,Armenia,18 years,Girls,2007,1
280142,Aabenraa,Persons of Danish origin,Denmark,Denmark,18 years,Boys,2007,2
296304,Aabenraa,Persons of Danish origin,Denmark,Denmark,18 years,Girls,2007,2
300536,Aabenraa,Immigrants,Iraq,Iraq,18 years,Boys,2007,1


## Explore each data set

# Merge data sets

# Analysis

# Conclusion